In [48]:
using Distributed
using Base.Threads
@everywhere begin
    using BenchmarkTools
    using ParallelTemperingMonteCarlo
end
#addprocs(nthreads())

In order to use a function of several variables in the pmap environment we require a curry function

In [49]:
@everywhere curry(f,y) = x -> f(x,y)
@everywhere add_xy(x,y) = x + y 


In [50]:
pmap(curry(add_xy,10), 1:5)


5-element Vector{Int64}:
 11
 12
 13
 14
 15

NB spawning processes is expensive and _not recommended at all_ for simple loops and functions.

# Sync macros

sync requires all tasks inside to complete before moving on, async moves right on along without waiting, 

In [51]:
@sync begin
    sleep(2)
    println("slept for two")

    @async begin 
        sleep(5)
        println("nice and rested")
    end
#the async wrapper skips straight to done
    println("done")
end

We'll simulate a complex process with a 2 second sleep

In [52]:
function simtest(x)
    sleep(2)
    return x
end

simtest (generic function with 1 method)

In [53]:
@time begin 
    veccy = []
    for i = 1:10

        y = simtest(i)
        push!(veccy,y)
        
    end
    println(veccy)
end

@time begin 
    @sync for i = 1:10
        veccy = []
        @async begin 
            y = simtest(i)
            push!(veccy,y)
        end
    end
    println(veccy)
end

NB: Asynchronous tasks are not parallel, demonstrated above they do boost performance for large operations. Use @spawn not @async for parallel operations

Let's test a data unsafe operation

In [54]:
function printtest(x::Bool = true)
    println("begin")
    if x == true
        @sync for i=1:10
            @spawn println("$i $i $i $i $i ")
        end
    else
        for i=1:10
            println("$i $i $i $i $i ")
        end
    end
    println("end")
end

printtest (generic function with 2 methods)

In [56]:
@time printtest()
println("done")

@time printtest(false)
println("done")
# @time begin 
#     println("begin")
#     @threads for i=1:10
#         println("$i $i $i $i $i ")
#     end
#     println("end")
# end

# NB 
Categorically do not use @spawn where data ordering is relevant. Additionally, it is considered bad practice to parallelise any operation faster than 100 $\mu m$ for threads or 100ms for spawning as it does not increase speed enough to compensate for the time taken to spawn the operation.

Below we test specific functions for this purpose.

## Onwards, time-testing some functions

In [57]:
n_atoms = 13

# temperature grid
ti = 5.
tf = 16.
n_traj = 32

temp = TempGrid{n_traj}(ti,tf) 

# MC simulation details
mc_cycles = 300000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles

#move_atom=AtomMove(n_atoms) #move strategy (here only atom moves, n_atoms per MC cycle)
displ_atom = 0.1 # Angstrom
n_adjust = 100

max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]

mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)

#moves - allowed at present: atom, volume and rotation moves (volume,rotation not yet implemented)
move_strat = MoveStrategy(atom_moves = n_atoms)  

#ensemble
ensemble = NVT(n_atoms)

#ELJpotential for neon
#c1=[-10.5097942564988, 0., 989.725135614556, 0., -101383.865938807, 0., 3918846.12841668, 0., -56234083.4334278, 0., 288738837.441765]
#elj_ne1 = ELJPotential{11}(c1)

c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

#starting configurations
#icosahedral ground state of Ne13 (from Cambridge cluster database) in Angstrom
pos_ne13 = [[2.825384495892464, 0.928562467914040, 0.505520149314310],
[2.023342172678102,	-2.136126268595355, 0.666071287554958],
[2.033761811732818,	-0.643989413759464, -2.133000349161121],
[0.979777205108572,	2.312002562803556, -1.671909307631893],
[0.962914279874254,	-0.102326586625353, 2.857083360096907],
[0.317957619634043,	2.646768968413408, 1.412132053672896],
[-2.825388342924982, -0.928563755928189, -0.505520471387560],
[-0.317955944853142, -2.646769840660271, -1.412131825293682],
[-0.979776174195320, -2.312003751825495, 1.671909138648006],
[-0.962916072888105, 0.102326392265998,	-2.857083272537599],
[-2.023340541398004, 2.136128558801072,	-0.666071089291685],
[-2.033762834001679, 0.643989905095452, 2.132999911364582],
[0.000002325340981,	0.000000762100600, 0.000000414930733]]

#convert to Bohr
AtoBohr = 1.8897259886
pos_ne13 = pos_ne13 * AtoBohr

length(pos_ne13) == n_atoms || error("number of atoms and positions not the same - check starting config")

#boundary conditions 
bc_ne13 = SphericalBC(radius=5.32*AtoBohr)   #5.32 Angstrom

#starting configuration
start_config = Config(pos_ne13, bc_ne13)

#histogram information
n_bin = 100
#en_min = -0.006    #might want to update after equilibration run if generated on the fly
#en_max = -0.001    #otherwise will be determined after run as min/max of sampled energies (ham vector)

#construct array of MCState (for each temperature)
mc_states = [MCState(temp.t_grid[i], temp.beta_grid[i], start_config, pot; max_displ=[max_displ_atom[i],0.01,1.]) for i in 1:n_traj]

#results = Output(n_bin, max_displ_vec)
results = Output{Float64}(n_bin; en_min = mc_states[1].en_tot)


Output{Float64}(100, 0.0, 0.0, Float64[], Float64[], Float64[], Vector{Float64}[], Float64[], Float64[], Float64[], Float64[], Float64[])

above we just define the 13 atom system, below we show that threading halves the time taken to complete one mc_step per trajectory.

In [97]:
@benchmark begin 
    for i in 1:mc_params.n_traj
        mc_step!(mc_states[i],pot,ensemble,1,0,0);
    end
end



BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  13.416 μs …  2.364 ms  ┊ GC (min … max): 0.00% … 97.15%
 Time  (median):     15.514 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   17.374 μs ± 39.790 μs  ┊ GC (mean ± σ):  3.90% ±  1.69%

   ▃  ▁ ▁▄▇█▁                                                  
  ▃██████████▅▅▄▃▂▂▂▃▃▂▂▃▇▇▅▆▅▅▄▄▃▃▃▃▂▂▂▂▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁ ▃
  13.4 μs         Histogram: frequency by time        25.8 μs <

 Memory estimate: 13.09 KiB, allocs estimate: 98.

In [98]:
@benchmark begin 
    @threads for i in 1:mc_params.n_traj
        mc_step!(mc_states[i],pot,ensemble,1,0,0);
    end
end


BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   6.717 μs …  5.594 ms  ┊ GC (min … max): 0.00% … 99.38%
 Time  (median):     10.815 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   14.329 μs ± 80.327 μs  ┊ GC (mean ± σ):  7.66% ±  1.40%

  ▇▇▅▇█▇▅▄▂▂▁▁                                                ▂
  █████████████▇▇▇▇▆▆▆▆▇▆▆▆▅▆▅▄▄▄▅▄▄▄▃▆▅▇▆▇▆▇▇▇▆▇▆▅▃▄▄▃▅▅▄▃▃▃ █
  6.72 μs      Histogram: log(frequency) by time      65.3 μs <

 Memory estimate: 15.73 KiB, allocs estimate: 106.

In [99]:
@benchmark begin 
    for i in 1:mc_params.n_traj
        x = MCRun.atom_displacement(mc_states[i].config.pos[2],mc_states[i].max_displ[1],mc_states[i].config.bc);
    end
end


BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  10.669 μs …  3.255 ms  ┊ GC (min … max): 0.00% … 98.72%
 Time  (median):     12.217 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   13.919 μs ± 32.663 μs  ┊ GC (mean ± σ):  2.31% ±  0.99%

  ▆██▇▆▆▆▅▆▆▅▅▄▃▃▁▁                              ▁▂▂ ▁▁▁      ▂
  ███████████████████▇▅▇▆██▇▇▇▇▇█▇▇█▇█▇▆▅▅▆▇▆█▇▇███████████▇█ █
  10.7 μs      Histogram: log(frequency) by time      28.3 μs <

 Memory estimate: 8.09 KiB, allocs estimate: 290.

In [100]:
@benchmark begin 
    @threads for i in 1:mc_params.n_traj
        x = MCRun.atom_displacement(mc_states[i].config.pos[2],mc_states[i].max_displ[1],mc_states[i].config.bc);
    end
end

BenchmarkTools.Trial: 10000 samples with 4 evaluations.
 Range (min … max):   6.494 μs …  3.794 ms  ┊ GC (min … max): 0.00% … 98.99%
 Time  (median):      9.929 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   15.138 μs ± 78.166 μs  ┊ GC (mean ± σ):  9.73% ±  1.98%

  ██▆██▆▄▂▁▁                           ▁▁▁▁▁▁                 ▂
  ████████████▇▇▇▇▇▆▆▆▆▆▆▆▅▅▅▆▅▄▅▅▅▆▆▇██████████▇▇▆▄▅▅▅▅▆▅▅▃▅ █
  6.49 μs      Histogram: log(frequency) by time      63.3 μs <

 Memory estimate: 10.73 KiB, allocs estimate: 299.

There is a speed-up in calculation for the atom_displacement step, but the greatest increase is to complete the metropolis condition as well, so threading works perfectly for the dimer by simply threading the loops. 

# Time to test the writing step

Let's test printing 32 (traj) iterations of 55(atoms) and time this with the @sync @sync vs without

In [101]:
t1 = @benchmark begin
    filetest = open("test.dat", "w+")
    
    @sync begin
    @async for i=1:32
         @async for j=1:55
            write(filetest,"$i $j \n")
        end
    end
end

close(filetest)
end

BenchmarkTools.Trial: 6244 samples with 1 evaluation.
 Range (min … max):  568.925 μs …  10.744 ms  ┊ GC (min … max): 0.00% …  0.00%
 Time  (median):     686.883 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   790.211 μs ± 583.668 μs  ┊ GC (mean ± σ):  8.06% ± 10.01%

  ▅█▆▄▃▂▁▁▁▁ ▁                                                  ▁
  █████████████▆▆▃▃▁▁▃▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▃▃▅▇▆ █
  569 μs        Histogram: log(frequency) by time       4.56 ms <

 Memory estimate: 649.17 KiB, allocs estimate: 12513.

In [102]:
t2 = @benchmark begin
    filetest = open("test2.dat", "w+")
    
    begin
    for i=1:32
        for j=1:55
            write(filetest,"$i $j \n")
        end
    end
end

    close(filetest)
end

BenchmarkTools.Trial: 6943 samples with 1 evaluation.
 Range (min … max):  556.888 μs …  14.114 ms  ┊ GC (min … max): 0.00% … 0.00%
 Time  (median):     639.265 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   712.756 μs ± 539.597 μs  ┊ GC (mean ± σ):  8.06% ± 9.96%

  ▇█▅▃▁▁                                                        ▁
  ████████▇▆▄▁▃▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▇ █
  557 μs        Histogram: log(frequency) by time       4.37 ms <

 Memory estimate: 633.23 KiB, allocs estimate: 12332.

It's slower to use the sync async functionality, but the atom-invariance within trajectories may make @spawn or @threads more useful

In [89]:
t3 = @benchmark begin
    filetest = open("test3.dat", "w+")
   
   @sync begin

   @async for i=1:32
        @threads for j=1:55
           write(filetest,"$i $j \n")
       end
   end
end

close(filetest)
end

BenchmarkTools.Trial: 1146 samples with 1 evaluation.
 Range (min … max):  1.847 ms … 58.466 ms  ┊ GC (min … max): 0.00% … 95.29%
 Time  (median):     3.064 ms              ┊ GC (median):    0.00%
 Time  (mean ± σ):   4.311 ms ±  4.282 ms  ┊ GC (mean ± σ):  4.85% ±  5.97%

  ▆▇█▇▆▆▅▅▄▃▃▃▁▂▁▁▃▁▁▁▁▁                                     ▁
  ████████████████████████▇█▅▇█▆█▆▇▅▇▆▆▆▅▆▇▆▆▇▅▄▅▄▁▁▅▆▁▁▁▄▁▅ █
  1.85 ms      Histogram: log(frequency) by time     15.8 ms <

 Memory estimate: 755.09 KiB, allocs estimate: 13787.

In [90]:
t4 = @benchmark begin
    filetest = open("test4.dat", "w+")
   
   @sync begin

   @async for i=1:32
        for j=1:55
        @spawn write(filetest,"$i $j \n")
       end
   end
end

close(filetest)
end

BenchmarkTools.Trial: 237 samples with 1 evaluation.
 Range (min … max):   6.170 ms … 169.256 ms  ┊ GC (min … max):  0.00% … 56.55%
 Time  (median):     11.096 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   21.665 ms ±  34.822 ms  ┊ GC (mean ± σ):  26.76% ± 14.44%

  ██▆▄▃▂▂                                                       
  ████████▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▆▄▇▄▄▁▁▁▄▆▄▄▄▁▄ ▆
  6.17 ms       Histogram: log(frequency) by time       165 ms <

 Memory estimate: 3.07 MiB, allocs estimate: 61638.

Nope. Writing shouldn't be parallelised. This isn't advantageous. Atom moves on the otherhand benefit from this. Operations, and in particular for-loops for the storage steps are next on the agenda